<a href="https://colab.research.google.com/github/vishwajeet1024/Practice-Projects/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imorting the Dependencies


In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


Data Loading and processing

In [36]:
# loading data in pandas dataFrame
credit_card_data = pd.read_csv('/content/drive/MyDrive/Dataset/creditcard.csv')

In [37]:
# first 5 rows of the dataset
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [38]:
# Checking Dimentions of the dataset
credit_card_data.shape

(284807, 31)

In [39]:
# chcking some information about dataset
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [40]:
# checking for missing value
credit_card_data.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [41]:
# checking the legit transactions & fraudulent transation
credit_card_data['Class'].value_counts()

,count
Class,
0,284315
1,492


This data is very unbalenced


0 --> Normal Transaction

1 --> fraudulent transaction

In [42]:
# seperating the data for the analysis
legit = credit_card_data[credit_card_data['Class'] == 0]
fraud = credit_card_data[credit_card_data['Class'] ==1]

print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [43]:
# checking statical measure of the data
legit.Amount.describe()

,Amount
count,284315.000000
mean,88.291022
std,250.105092
min,0.000000
25%,5.650000
50%,22.000000
75%,77.050000
max,25691.160000


In [44]:
fraud.Amount.describe()

,Amount
count,492.000000
mean,122.211321
std,256.683288
min,0.000000
25%,1.000000
50%,9.250000
75%,105.890000
max,2125.870000


In [45]:
# compare the value for both the transaction
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Under-Sampling
<br>
<br>

Build a sample dataset containingf similar distribution of normal transaction and Fraudulent Transactiobnn
<br>

Number of Fraudulent transation --> 492

In [46]:
# taking equavelent sample from legit
legit_sample = legit.sample(n=492)

In [47]:
# concading the legit_sample and fraud datasets
new_dataset =pd.concat([legit_sample, fraud], axis =0)

In [48]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
93861,64618.0,1.246130,0.702325,0.459483,2.267243,0.318975,-0.026837,0.205176,-0.130511,-0.899762,0.629698,-0.984123,0.453793,1.652663,-0.054090,0.460385,0.950533,-1.036627,-0.378010,-0.615700,-0.033083,-0.287742,-0.830258,0.024293,-0.474598,0.409563,-0.218451,0.001867,0.023181,2.62,0
148344,89734.0,1.835842,-0.435503,-0.237230,1.450980,-0.712061,-0.309968,-0.511040,0.001118,1.191007,0.144077,-1.314835,0.056211,-0.371704,-0.082245,0.329060,0.324795,-0.638894,0.373962,-0.576128,-0.159777,0.224451,0.728343,0.051136,-0.124747,-0.062847,-0.528255,0.050230,-0.022080,67.70,0
12146,21154.0,1.210991,0.753194,0.127593,2.502477,0.434070,-0.297114,0.315559,-0.154075,0.101255,0.556619,1.697552,-2.428786,0.858683,2.273877,-1.128802,0.625731,-0.073759,0.462776,-0.672266,-0.231887,-0.143390,-0.256663,-0.137216,-0.065928,0.696261,0.034811,-0.075553,-0.011200,3.65,0
226033,144478.0,-1.263823,0.188053,0.436852,-2.754139,0.072831,0.360503,-0.405261,0.624819,-0.255516,0.733704,-0.526069,-1.184463,-1.294245,-0.224958,-0.886493,2.041918,-0.699326,-0.375919,0.620929,0.291942,-0.147515,-0.319049,0.060186,-0.439231,-0.175569,-0.573201,0.269735,0.082108,8.99,0
90267,62944.0,1.497944,-0.246422,-0.483434,-0.834949,-0.202680,-0.963933,0.017334,-0.413198,-1.234569,0.661397,-0.729317,-0.394523,1.114257,-0.101197,0.381043,0.897738,0.094449,-1.470689,0.902360,0.147020,0.153420,0.420765,-0.265149,-0.261698,0.918821,-0.043465,-0.024428,-0.003230,15.00,0


In [49]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,-5.587794,2.115795,-5.417424,-1.235123,-6.665177,0.401701,-2.897825,-4.570529,-1.315147,0.391167,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,-3.232153,2.858466,-3.096915,-0.792532,-5.210141,-0.613803,-2.155297,-3.267116,-0.688505,0.737657,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,-3.463891,1.794969,-2.775022,-0.418950,-4.057162,-0.712616,-1.603015,-5.035326,-0.507000,0.266272,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,-5.245984,1.933520,-5.030465,-1.127455,-6.416628,0.141237,-2.549498,-4.614717,-1.478138,-0.035480,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,-0.888722,0.491140,0.728903,0.380428,-1.948883,-0.832498,0.519436,0.903562,1.197315,0.593509,-0.017652,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [50]:
new_dataset['Class'].value_counts()

,count
Class,
0,492
1,492


In [51]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,98191.664634,0.109578,-0.050132,-0.072773,0.011656,-0.018999,0.004336,0.105345,-0.007416,0.025109,-0.001453,0.013249,-0.042079,-0.046333,0.019648,0.033723,0.053749,0.035405,-0.075434,0.061608,-0.013813,-0.029944,-0.020580,0.009577,-0.002614,-0.000886,-0.043745,-0.012637,-0.016646,98.042134
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Spliting the data info Features & Targets

In [52]:
X = new_dataset.drop(columns = 'Class', axis=1)
Y = new_dataset['Class']

In [53]:
print(X)
print("----------------------------------")
print(Y)

            Time        V1        V2  ...       V27       V28  Amount
93861    64618.0  1.246130  0.702325  ...  0.001867  0.023181    2.62
148344   89734.0  1.835842 -0.435503  ...  0.050230 -0.022080   67.70
12146    21154.0  1.210991  0.753194  ... -0.075553 -0.011200    3.65
226033  144478.0 -1.263823  0.188053  ...  0.269735  0.082108    8.99
90267    62944.0  1.497944 -0.246422  ... -0.024428 -0.003230   15.00
...          ...       ...       ...  ...       ...       ...     ...
279863  169142.0 -1.927883  1.125653  ...  0.292680  0.147968  390.00
280143  169347.0  1.378559  1.289381  ...  0.389152  0.186637    0.76
280149  169351.0 -0.676143  1.126366  ...  0.385107  0.194361   77.89
281144  169966.0 -3.113832  0.585864  ...  0.884876 -0.253700  245.00
281674  170348.0  1.991976  0.158476  ...  0.002988 -0.015309   42.53

[984 rows x 30 columns]
----------------------------------
93861     0
148344    0
12146     0
226033    0
90267     0
         ..
279863    1
280143    1
2801

Split the data into Training and the testing data

In [54]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size =0.2, random_state =2)

In [55]:
# checking dimention of train and test data
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


Model training       

Logistic Regression

In [56]:
model = LogisticRegression()

In [57]:
# training the model
model.fit(X_train, Y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Model Evaluation


Accuracy Score

In [58]:
# checking accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [59]:
print('Accuracy on Trainng data : ', training_data_accuracy)

Accuracy on Trainng data :  0.9415501905972046


In [60]:
# checking accuracy on testing data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [61]:
print("accuracy on test data : ", testing_data_accuracy)

accuracy on test data :  0.934010152284264
